# Decoding with ANOVA + SVM: Chocolate Data -- milkshake (small subset)
### This is a simple decoding using feature selection followed by an SVM.  
Reference: http://nilearn.github.io/auto_examples/02_decoding/plot_haxby_anova_svm.html#sphx-glr-auto-examples-02-decoding-plot-haxby-anova-svm-py

In [3]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
import pandas as pd 
from nilearn.input_data import NiftiMasker 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')

## Load our dataset (concatenated image), image mask (power roi 4b) and our behavioral data

### We select our target paradigm, here we are doing h2O vs LF_LS

In [ ]:
#image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/power_roimask_4bi.nii.gz'
#our behavioral csv file 
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/w1_milkshake_48.csv'
#our dataset concatenated image 
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/w1_milkshake_48.nii.gz'
#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")
#grab conditional labels 
y = behavioral["Label"]



In [ ]:
#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['LF_LS_receipt', "h20_receipt"])
y = y[condition_mask]
#confirm we have the # of condtions needed
print(y.unique())

['h20_receipt' 'LF_HS_receipt']

In [ ]:

# Record these as an array of sessions, with fields
session = behavioral[condition_mask].to_records(index=False)
print(session.dtype.names)

('Label', 'sess')

In [ ]:
# for decoding, standardizing the data is often very important  
# if data hasn't been smoothed, you can do that here too  
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(dataset)
#Apply our condition mask
X=X[condition_mask]
session=session[condition_mask]

### Here we are settting up the SVM model, starting with a linear kernel 

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')


# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test,
# namely Anova. We set the number of features to be selected to 500
from sklearn.feature_selection import SelectKBest, f_classif
feature_selection = SelectKBest(f_classif, k=500)

# We have our classifier (SVC), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

### Here we are going to compute the prediction scores using cross-validation

In [ ]:
anova_svc.fit(X, conditions)
y_pred = anova_svc.predict(X)

from sklearn.model_selection import cross_val_score

k_range = [10, 15, 30, 50, 150, 300, 500, 1000, 1500, 3000, 5000]
cv_scores = []
scores_validation = []

for k in k_range:
    feature_selection.k = k
    cv_scores.append(np.mean(
        cross_val_score(anova_svc, X[session < 1], conditions[session < 1], cv=3)))
    print("CV score: %.4f" % cv_scores[-1])

    anova_svc.fit(X[session < 1], conditions[session < 1])
    y_pred = anova_svc.predict(X[session == 1])
    scores_validation.append(np.mean(y_pred == conditions[session == 1]))
    print("score validation: %.4f" % scores_validation[-1])

### Now we are using nested cross-validation 

In [ ]:
from sklearn.model_selection import cross_val_score

k_range = [10, 50, 150, 300, 500, 1000,3000]
from sklearn.model_selection import GridSearchCV
# We are going to tune the parameter 'k' of the step called 'anova' in
# the pipeline. Thus we need to address it as 'anova__k'.

# Note that GridSearchCV takes an n_jobs argument that can make it go
# much faster
grid = GridSearchCV(anova_svc, param_grid={'anova__k': k_range}, verbose=1,
                    cv=3, n_jobs=4)
nested_cv_scores = cross_val_score(grid, X, y, cv=3)

print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

Fitting 3 folds for each of 7 candidates, totalling 21 fits  
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.  
[Parallel(n_jobs=4)]: Done  21 out of  21 | elapsed: 13.5min finished  
Fitting 3 folds for each of 7 candidates, totalling 21 fits  
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.  
[Parallel(n_jobs=4)]: Done  21 out of  21 | elapsed: 13.3min finished  
Fitting 3 folds for each of 7 candidates, totalling 21 fits  
[Parallel(n_jobs=4)]: Using backend SequentialBackend with 1 concurrent workers.  
[Parallel(n_jobs=4)]: Done  21 out of  21 | elapsed: 13.4min finished  
Nested CV score: 0.5025

### Visualize and save our results

In [ ]:
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(func_filename)

# Create the figure
from nilearn.plotting import plot_stat_map, show
plot_stat_map(weight_img, mean_img, title='SVM weights')

# Saving the results as a Nifti file may also be important
weight_img.to_filename('milkshake_svm_nested_48.nii')